## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [2]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [3]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [4]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsRsast'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)

In [5]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['SmoothSubspace', 'GunPoint', 'ItalyPowerDemand', 'ShapeletSim',
       'Chinatown', 'Coffee', 'BirdChicken', 'SyntheticControl'],
      dtype=object)

In [6]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [7]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.900000
1,Chinatown,0.950865,0.962177,0.938503,0.960233,0.947833,0.963460,Chinatown,0.982507
2,Coffee,0.937143,0.997143,0.951429,0.937143,0.994286,0.992857,Coffee,1.000000
3,GunPoint,0.778933,0.971111,0.784889,0.947556,0.918311,0.964978,GunPoint,1.000000
4,ItalyPowerDemand,0.890897,0.903026,0.879585,0.874921,0.885079,0.897933,ItalyPowerDemand,0.968902
5,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
6,SmoothSubspace,0.816000,0.863022,0.821333,0.849511,0.827556,0.861689,SmoothSubspace,0.980000
7,SyntheticControl,0.358533,0.318400,0.311067,0.292400,0.292800,0.352933,SyntheticControl,1.000000


In [8]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.769476,0.857328,0.844059,0.863697,0.857603
ACF&PACF,0.844396,0.885622,0.880608,0.892317,0.891222
Max ACF,0.740634,0.828525,0.823008,0.852646,0.845822
Max PACF,0.765287,0.843352,0.835721,0.848768,0.855867
None,0.753958,0.816807,0.802488,0.821473,0.816074
PACF,0.839971,0.878166,0.889892,0.890193,0.893048


In [9]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.755533,0.839602,0.848180,0.876675,0.872173
ACF&PACF,0.839703,0.869130,0.894631,0.896749,0.893954
Max ACF,0.711772,0.833880,0.826479,0.856589,0.861916
Max PACF,0.743105,0.841649,0.835368,0.866184,0.862689
None,0.705683,0.815405,0.816517,0.833425,0.839770
PACF,0.837010,0.872620,0.894034,0.895440,0.892166


#### Time: Subsequence Lenght Method 

In [10]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SyntheticControl
len_method,,,,,,,,
ACF,81.630738,0.143859,29.515313,39.144550,0.826132,7.645123,2.081739,128.364227
ACF&PACF,297.280662,0.444094,102.533573,88.200498,2.032732,222.211859,3.407822,410.825064
Max ACF,2.745482,0.145839,2.160797,4.153232,0.839920,2.640422,2.087500,45.500390
Max PACF,107.885490,0.162721,6.330146,3.936128,0.915300,124.217464,2.404325,30.050111
None,1.853694,0.133453,1.236922,2.529203,0.829915,1.836391,2.064745,27.938759
PACF,208.928276,0.387296,74.365190,48.907430,2.043757,215.296044,3.382209,261.462549


In [11]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,2.306491,23.546900,34.347834,38.959400,38.913366
ACF&PACF,11.692671,113.741273,142.809410,160.487941,159.791614
Max ACF,0.318923,2.437185,4.947424,6.855869,6.815192
Max PACF,4.328097,42.916742,51.452048,45.560316,44.478932
None,0.242490,1.663898,3.331226,4.433133,4.558724
PACF,8.963710,90.390296,108.834168,119.087090,115.890431


In [12]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.834816,7.379167,21.693022,36.024354,72.142631
ACF&PACF,35.642166,54.674590,100.235848,148.155941,249.814365
Max ACF,0.211590,1.208360,3.349217,5.608288,10.997138
Max PACF,32.585197,32.325037,38.952350,40.958794,43.914757
None,0.143882,0.811427,2.219310,3.709480,7.345371
PACF,34.559217,45.851549,81.355164,108.666038,172.733727


#### Variance: Subsequence Lenght Method 

In [13]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.036464,0.031429,0.031073,0.016943,0.022186
ACF&PACF,0.028261,0.028787,0.019847,0.020082,0.023514
Max ACF,0.039814,0.029019,0.040330,0.028845,0.019296
Max PACF,0.043850,0.033723,0.039345,0.024784,0.025307
None,0.046292,0.039136,0.040360,0.035401,0.035655
PACF,0.025197,0.027077,0.018197,0.020642,0.022682


In [14]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.037563,0.024790,0.030501,0.020836,0.027859
ACF&PACF,0.033316,0.021764,0.025686,0.020396,0.020191
Max ACF,0.042755,0.033031,0.039183,0.023739,0.025598
Max PACF,0.045508,0.031395,0.037172,0.030864,0.026791
None,0.050305,0.038311,0.044994,0.035418,0.036883
PACF,0.029805,0.025854,0.020149,0.020938,0.017562


#### Another Results

In [15]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,3000,3000,3000
unique,8,6,150
top,SmoothSubspace,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=1
freq,450,500,20


In [16]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
Coffee,1.500986
GunPoint,3.893147
ItalyPowerDemand,0.155995
ShapeletSim,11.955152
SmoothSubspace,0.321424
SyntheticControl,6.278758


In [17]:
df_result['time'].sum()/(60*60)

38.72507758842574

#### Generate CD Diagram Tunning Hyperparameter

In [18]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [19]:
from cd_diagram.cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='cd-diagram_overall_tunning.png')
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)


['ACF&PACF: n_random_points=1 nb_inst_per_class=1'
 'ACF&PACF: n_random_points=1 nb_inst_per_class=10'
 'ACF&PACF: n_random_points=1 nb_inst_per_class=100'
 'ACF&PACF: n_random_points=1 nb_inst_per_class=30'
 'ACF&PACF: n_random_points=1 nb_inst_per_class=50'
 'ACF&PACF: n_random_points=10 nb_inst_per_class=1'
 'ACF&PACF: n_random_points=10 nb_inst_per_class=10'
 'ACF&PACF: n_random_points=10 nb_inst_per_class=100'
 'ACF&PACF: n_random_points=10 nb_inst_per_class=30'
 'ACF&PACF: n_random_points=10 nb_inst_per_class=50'
 'ACF&PACF: n_random_points=100 nb_inst_per_class=1'
 'ACF&PACF: n_random_points=100 nb_inst_per_class=10'
 'ACF&PACF: n_random_points=100 nb_inst_per_class=100'
 'ACF&PACF: n_random_points=100 nb_inst_per_class=30'
 'ACF&PACF: n_random_points=100 nb_inst_per_class=50'
 'ACF&PACF: n_random_points=30 nb_inst_per_class=1'
 'ACF&PACF: n_random_points=30 nb_inst_per_class=10'
 'ACF&PACF: n_random_points=30 nb_inst_per_class=100'
 'ACF&PACF: n_random_points=30 nb_inst_per_cla

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


ACF&PACF: n_random_points=1 nb_inst_per_class=1        0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=10       0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=100      0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=30       0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=50       0.0
ACF&PACF: n_random_points=10 nb_inst_per_class=1       0.0
ACF&PACF: n_random_points=10 nb_inst_per_class=10      0.0
ACF&PACF: n_random_points=10 nb_inst_per_class=100     0.0
ACF&PACF: n_random_points=10 nb_inst_per_class=30      0.0
ACF&PACF: n_random_points=10 nb_inst_per_class=50      0.0
ACF&PACF: n_random_points=100 nb_inst_per_class=1      0.0
ACF&PACF: n_random_points=100 nb_inst_per_class=10     0.0
ACF&PACF: n_random_points=100 nb_inst_per_class=100    0.0
ACF&PACF: n_random_points=100 nb_inst_per_class=30     1.0
ACF&PACF: n_random_points=100 nb_inst_per_class=50     0.0
ACF&PACF: n_random_points=30 nb_inst_per_class=1       0.0
ACF&PACF: n_random_points=30 nb_inst_per_class=10      0

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


PACF: n_random_points=1 nb_inst_per_class=1        0.0
PACF: n_random_points=1 nb_inst_per_class=10       0.0
PACF: n_random_points=1 nb_inst_per_class=100      0.0
PACF: n_random_points=1 nb_inst_per_class=30       0.0
PACF: n_random_points=1 nb_inst_per_class=50       0.0
PACF: n_random_points=10 nb_inst_per_class=1       0.0
PACF: n_random_points=10 nb_inst_per_class=10      0.0
PACF: n_random_points=10 nb_inst_per_class=100     0.0
PACF: n_random_points=10 nb_inst_per_class=30      0.0
PACF: n_random_points=10 nb_inst_per_class=50      1.0
PACF: n_random_points=100 nb_inst_per_class=1      1.0
PACF: n_random_points=100 nb_inst_per_class=10     0.0
PACF: n_random_points=100 nb_inst_per_class=100    0.0
PACF: n_random_points=100 nb_inst_per_class=30     0.0
PACF: n_random_points=100 nb_inst_per_class=50     0.0
PACF: n_random_points=30 nb_inst_per_class=1       1.0
PACF: n_random_points=30 nb_inst_per_class=10      1.0
PACF: n_random_points=30 nb_inst_per_class=100     0.0
PACF: n_ra

...

Max PACF: n_random_points=1 nb_inst_per_class=1        1.0
Max PACF: n_random_points=1 nb_inst_per_class=10       0.0
Max PACF: n_random_points=1 nb_inst_per_class=100      0.0
Max PACF: n_random_points=1 nb_inst_per_class=30       0.0
Max PACF: n_random_points=1 nb_inst_per_class=50       0.0
Max PACF: n_random_points=10 nb_inst_per_class=1       0.0
Max PACF: n_random_points=10 nb_inst_per_class=10      0.0
Max PACF: n_random_points=10 nb_inst_per_class=100     0.0
Max PACF: n_random_points=10 nb_inst_per_class=30      0.0
Max PACF: n_random_points=10 nb_inst_per_class=50      0.0
Max PACF: n_random_points=100 nb_inst_per_class=1      0.0
Max PACF: n_random_points=100 nb_inst_per_class=10     1.0
Max PACF: n_random_points=100 nb_inst_per_class=100    0.0
Max PACF: n_random_points=100 nb_inst_per_class=30     0.0
Max PACF: n_random_points=100 nb_inst_per_class=50     0.0
Max PACF: n_random_points=30 nb_inst_per_class=1       0.0
Max PACF: n_random_points=30 nb_inst_per_class=10      0

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


ACF&PACF: n_random_points=1 nb_inst_per_class=1      0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=10     0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=100    0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=30     0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=50     0.0
                                                    ... 
PACF: n_random_points=50 nb_inst_per_class=1         0.0
PACF: n_random_points=50 nb_inst_per_class=10        0.0
PACF: n_random_points=50 nb_inst_per_class=100       0.0
PACF: n_random_points=50 nb_inst_per_class=30        0.0
PACF: n_random_points=50 nb_inst_per_class=50        0.0
Length: 150, dtype: float64
Max ACF: n_random_points=10 nb_inst_per_class=1       132.4375
Max ACF: n_random_points=1 nb_inst_per_class=1        132.0000
Max ACF: n_random_points=30 nb_inst_per_class=1       127.5000
Max PACF: n_random_points=30 nb_inst_per_class=1      126.3125
Max PACF: n_random_points=10 nb_inst_per_class=1      126.2500
                              

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


ACF&PACF: n_random_points=1 nb_inst_per_class=1      0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=10     0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=100    0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=30     0.0
ACF&PACF: n_random_points=1 nb_inst_per_class=50     0.0
                                                    ... 
PACF: n_random_points=50 nb_inst_per_class=1         0.0
PACF: n_random_points=50 nb_inst_per_class=10        0.0
PACF: n_random_points=50 nb_inst_per_class=100       0.0
PACF: n_random_points=50 nb_inst_per_class=30        0.0
PACF: n_random_points=50 nb_inst_per_class=50        0.0
Length: 150, dtype: float64


In [21]:
average_ranks

,0
Max ACF: n_random_points=10 nb_inst_per_class=1,132.4375
Max ACF: n_random_points=1 nb_inst_per_class=1,132.0000
Max ACF: n_random_points=30 nb_inst_per_class=1,127.5000
Max PACF: n_random_points=30 nb_inst_per_class=1,126.3125
Max PACF: n_random_points=10 nb_inst_per_class=1,126.2500
...,...
PACF: n_random_points=100 nb_inst_per_class=50,42.1875
ACF&PACF: n_random_points=50 nb_inst_per_class=100,41.1875
ACF&PACF: n_random_points=30 nb_inst_per_class=50,40.5000
ACF&PACF: n_random_points=100 nb_inst_per_class=30,38.0625


In [22]:
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=100 nb_inst_per_class=50'], dtype='object')

In [ ]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

##### Overall time complexity

In [ ]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

In [ ]:
# create a pivot table with the mean of time spent in seconds by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')